In [ ]:
#violence model Run code

import numpy as np
import cv2
import time
from keras.models import load_model
from collections import deque

def print_results(limit=None):
    trueCount = 0
    imageSaved = 0
    filename = 'savedImage.jpg'
    finalImage = 'finaImage.jpg'
    sendAlert = 0
    location = "Bangalore"

    print("Loading model ...")
    model = load_model('/home//Downloads/modelnew.h5')
    Q = deque(maxlen=128)

    # Open the camera
    vs = cv2.VideoCapture(0)  # 0 corresponds to the default camera

    (W, H) = (None, None)
    count = 0

    while True:
        # read the next frame from the camera
        (grabbed, frame) = vs.read()

        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]
        label = i

        text_color = (0, 255, 0)  # default: green

        if label:  # Violence prob
            text_color = (0, 0, 255)  # red
            trueCount = trueCount + 1
        else:
            text_color = (0, 255, 0)

        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX

        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

        cv2.imshow("Live Video", output)

        if trueCount == 50:
            if imageSaved == 0:
                if label:
                    cv2.imwrite(filename, output)
                    imageSaved = 1

            if sendAlert == 0:
                timeMoment = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
                # Replace the following lines with your alert mechanism
                print(f"VIOLENCE ALERT!! \nLOCATION: {location} \nTIME: {timeMoment}")
                # Send photo if image is saved
                if label and imageSaved:
                    # Replace the following line with your alert mechanism
                    print("Sending photo...")
                sendAlert = 1

        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break

    print("[INFO] cleaning up...")
    vs.release()
    cv2.destroyAllWindows()

# Call the function
print_results()

In [ ]:
#Weapon run code

import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model
# from tensorflow.keras.preprocessing.image import img_to_array
# from tensorflow.keras.models import load_model

# Load the pre-trained image classification model
classification_model = load_model("/home//Documents/rajasthan hackthon/Weapon-Detection-And-Classification/models/model_latest.h5")

# Open a video capture object
cap = cv2.VideoCapture(0)  # 0 for the default camera

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Preprocess the frame for image classification
    resized_frame = cv2.resize(frame, (50, 50))
    resized_frame = img_to_array(resized_frame) / 255
    resized_frame = np.transpose(resized_frame, (2, 0, 1))
    resized_frame = np.reshape(resized_frame, (1, 3, 50, 50))

    # Make predictions using the image classification model
    classification_predictions = classification_model.predict(resized_frame)[0]
    print(classification_predictions)

    # Separate the three values
    value1, value2, value3 = classification_predictions

    # Determine the class based on conditions
    if value1 > value2 and value1 > value3:
#         if value1> float(5.0):
            label = "weapon detected"
    elif value2 > value1 and value2 > value3:
#         if value2>float(5.0):
            label = "none"
    elif value3>float(5.0):
        label = "weapon detected"
    else: 
        label = "weapon detected"

    # Display the frame with classification label
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Image Classification', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()

In [ ]:
#violence and Weapon combined

import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from collections import deque
import time

# Load the violence detection model
violence_model = load_model('/home//Downloads/modelnew.h5')
weapon_model = load_model("/home//Documents/rajasthan hackthon/Weapon-Detection-And-Classification/models/model_latest.h5")

Q = deque(maxlen=128)

trueCount = 0
imageSaved = 0
filename = 'savedImage.jpg'
sendAlert = 0
location = "Bangalore"

vs = cv2.VideoCapture(0)  # 0 corresponds to the default camera

(W, H) = (None, None)
count = 0

while True:
    (grabbed, frame) = vs.read()

    if W is None or H is None:
        (H, W) = frame.shape[:2]

    output = frame.copy()

    # Violence Detection
    frame_violence = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_violence = cv2.resize(frame_violence, (128, 128)).astype("float32")
    frame_violence = frame_violence.reshape(128, 128, 3) / 255

    preds_violence = violence_model.predict(np.expand_dims(frame_violence, axis=0))[0]
    Q.append(preds_violence)
    results_violence = np.array(Q).mean(axis=0)
    label_violence = (preds_violence > 0.50)[0]

    # Weapon Classification
    resized_frame = cv2.resize(frame, (50, 50))
    resized_frame = img_to_array(resized_frame) / 255
    resized_frame = np.transpose(resized_frame, (2, 0, 1))
    resized_frame = np.reshape(resized_frame, (1, 3, 50, 50))

    classification_predictions = weapon_model.predict(resized_frame)[0]

    value1, value2, value3 = classification_predictions

    if value1 > value2 and value1 > value3:
        label_weapon = "Weapon Detected"
    elif value2 > value1 and value2 > value3:
        label_weapon = "weapon Detected"
    elif value3 > 5.0:
        label_weapon = "Weapon Detected"
    else:
        label_weapon = "Weapon Detected"

    # Displaying Outputs
    text_violence = "Violence: {}".format(label_violence)
    text_weapon = "Weapon: {}".format(label_weapon)

    FONT = cv2.FONT_HERSHEY_SIMPLEX
    text_color_violence = (0, 0, 255) if label_violence else (0, 255, 0)
    text_color_weapon = (0, 255, 0) if label_weapon == "None" else (0, 0, 255)

    cv2.putText(output, text_violence, (35, 50), FONT, 1.25, text_color_violence, 3)
    cv2.putText(output, text_weapon, (35, 100), FONT, 1.25, text_color_weapon, 3)

    cv2.imshow("Combined Output", output)

    if label_violence:  # If violence detected
        trueCount += 1
        if trueCount == 50 and not imageSaved:
            cv2.imwrite(filename, output)
            imageSaved = 1

        if not sendAlert:
            timeMoment = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
            print(f"VIOLENCE ALERT!! \nLOCATION: {location} \nTIME: {timeMoment}")
            if imageSaved:
                print("Sending photo...")  # Replace with your alert mechanism
            sendAlert = 1

    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

print("[INFO] Cleaning up...")
vs.release()
cv2.destroyAllWindows()
